# Link Prediction w/ n2v

In [17]:
!pip install arxiv

     |████████████████████████████████| 81 kB 1.7 MB/s eta 0:00:01
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=9628adb2c5f911b2608828809f9555a106d3a0b745a02b4cf5c0b995c871b39d
  Stored in directory: /Users/vatsalpatel/Library/Caches/pip/wheels/83/63/2f/117884c3b19d46b64d3d61690333aa80c88dc14050e269c546
Successfully built sgmllib3k


In [1]:
import networkx as nx
import scipy as sp
import pandas as pd
import numpy as np
import arxiv

from node2vec import node2vec as n2v

In [3]:
# constants
queries = [
    'automl', 'machinelearning', 'data', 'phyiscs','mathematics', 'recommendation system', 'nlp', 'neural networks'
]

In [4]:
search = arxiv.Search(
  query = "automl",
  max_results = 3,
  sort_by = arxiv.SortCriterion.SubmittedDate,
  sort_order = arxiv.SortOrder.Descending
)

In [12]:
for result in search.results():
    print('Title: ', result.title, '\nDate: ',result.published , '\nId: ', result.entry_id,'\nURL: ', result.pdf_url, '\n\n', result.primary_category, result.categories)

Title:  Review of automated time series forecasting pipelines 
Date:  2022-02-03 17:26:27+00:00 
Id:  http://arxiv.org/abs/2202.01712v1 
URL:  http://arxiv.org/pdf/2202.01712v1 

 cs.LG ['cs.LG']
Title:  Hubble Asteroid Hunter: I. Identifying asteroid trails in Hubble Space Telescope images 
Date:  2022-02-01 06:56:20+00:00 
Id:  http://arxiv.org/abs/2202.00246v1 
URL:  http://arxiv.org/pdf/2202.00246v1 

 astro-ph.EP ['astro-ph.EP', 'astro-ph.IM']
Title:  NAS-Bench-Suite: NAS Evaluation is (Now) Surprisingly Easy 
Date:  2022-01-31 18:02:09+00:00 
Id:  http://arxiv.org/abs/2201.13396v1 
URL:  http://arxiv.org/pdf/2201.13396v1 

 cs.LG ['cs.LG', 'cs.AI', 'stat.ML']


In [2]:
# constants
edge_path = './data/edges.csv'
hero_path = './data/hero-network.csv'
node_path = './data/nodes.csv'
aven_path = './data/avengers.csv'
char_path = './data/characters.csv'

## Import Data

In [14]:
edge = pd.read_csv(edge_path)
hero = pd.read_csv(hero_path)
node = pd.read_csv(node_path)
char = pd.read_csv(char_path)
aven = pd.read_csv(aven_path)

In [16]:
char

,char,charname,birthname,types,universes,birthplace,superpowers,religions,gender,occupation,memberof
0,http://www.wikidata.org/entity/Q18754954,Silk,Cindy Moon,"mutate, comics character",Marvel Universe,NaN,"superhuman agility / reflexes, superhuman dura...",NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q3127430,Snowbird,Narya,"fictional deity, comics character",Marvel Universe,NaN,"flight, shapeshifting, superhuman strength, re...",NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q546340,Thunderbird,John Proudstar,"television character, animated character, mutant",Marvel Universe,NaN,"superhuman durability, superhuman speed, super...",NaN,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q1194236,Cannonball,Samuel Zachery Guthrie,"superhero film character, animated character, ...",Marvel Universe,NaN,thermo-chemical energy,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q2087326,Shriek,Frances Louise Barrison,"film character, comics character",Marvel Universe,NaN,sonic scream,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
132,http://www.wikidata.org/entity/Q2275950,Shatterstar,Gaveedra Seven,"superhero film character, mutant",Marvel Universe,NaN,"superhuman agility / reflexes, superhuman dura...",NaN,NaN,NaN,NaN
133,http://www.wikidata.org/entity/Q3453575,Rusty Collins,Russell Collins,"superhero film character, animated character, ...",Marvel Universe,NaN,pyrokinesis,NaN,NaN,NaN,NaN
134,http://www.wikidata.org/entity/Q865017,Mister Fantastic,Reed Richards,"film character, animated character, fictional ...",Marvel Universe,NaN,elasticity,NaN,NaN,NaN,NaN
135,http://www.wikidata.org/entity/Q616633,Betty Ross,Elizabeth Ross,"gamma mutate, harpy in a work of fiction, film...",Marvel Universe,NaN,"superhuman durability, superhuman speed, super...",NaN,NaN,NaN,NaN


## Generate Network